In [1]:
import numpy as np 

In [2]:
#例33.1

In [3]:
import numpy as np
#原始数据
original_data=[
               [1,0,5],[2,2,7],[3,6,1],[4,11,9],[5,12,2],[6,19,4],
               [7,22,3],[8,26,3],[9,36,1],[10,38,2],[11,45,5],
               [12,47,4],[13,49,1],[14,52,2],[15,61,1],[16,62,2],
               [17,65,1],[18,70,3],[19,72,4],[20,80,3],[21,81,2],
               [22,83,3],[23,86,6],[24,88,5],[25,92,1],[26,95,3],
               [27,101,2],[28,105,2],[29,106,1],[30,109,2],[31,114,1],
               [32,116,8],[33,117,4],[34,121,2],[35,127,1],[36,129,6],
               [37,130,3],[38,133,5],[39,135,2],[40,139,4],[41,142,1]
              ]
compute_data=[]
for i in range(len(original_data)):
    if i==0:
        compute_data.append([original_data[1][1]-original_data[0][1],0])
    elif i==len(original_data)-1:
        w=compute_data[i-1][1]+original_data[i-1][2]-compute_data[i-1][0]
        w=w if w>0 else 0
        compute_data.append([0,w])
    else:
        t=original_data[i+1][1]-original_data[i][1]
        w=compute_data[i-1][1]+original_data[i-1][2]-compute_data[i-1][0]
        w=w if w>0 else 0
        compute_data.append([t,w])
data=np.hstack((original_data,compute_data))
data[:5]

array([[ 1,  0,  5,  2,  0],
       [ 2,  2,  7,  4,  3],
       [ 3,  6,  1,  5,  6],
       [ 4, 11,  9,  1,  2],
       [ 5, 12,  2,  7, 10]])

In [4]:
aver_t=np.mean(data[:-1],axis=0)[3]
aver_s,aver_w=np.mean(data,axis=0)[[2,4]]
np.round((aver_t,aver_s,aver_w),3)

array([3.55 , 3.098, 3.317])

In [5]:
import sympy as sy
from sympy import series
t,lamda=sy.symbols('t lambda')
f=lambda t:sy.exp(lamda*t)
#so:1=exp(lamda*x)*exp(-lamda*x)
#display(f(t).series(t,0,6)*sy.exp(-lamda*t))
f(t).series(t,0,6)*sy.exp(-lamda*t)

(1 + lambda*t + lambda**2*t**2/2 + lambda**3*t**3/6 + lambda**4*t**4/24 + lambda**5*t**5/120 + O(t**6))*exp(-lambda*t)

In [6]:
#so:P(n>0)=1-exp(-lamda*t),即输入为泊松流和负指数分布是等价的

In [7]:
from scipy.stats import poisson,expon,uniform
import numpy as np

In [8]:
#例33.2

In [9]:
#生成服务对象达到时间
def gen_reach_T(lamda,uniform_scale,gen_size,random_seed):
    np.random.seed(random_seed)
    generator=poisson(lamda)
    nums=0
    gen_nums=[]
    while nums<gen_size:
        num=generator.rvs(size=1)
        gen_nums.append(num)
        nums+=num
    #修改最后一个元素，使列表gen_nums的各项和恰好为gen_size
    gen_nums[-1]-=nums-gen_size
    gen_nums=list(np.array(gen_nums).ravel())
    reach_T=[]
    for idx,num in enumerate(gen_nums):
        if num>0:
            T_s=uniform(idx,uniform_scale).rvs(size=num)
            T_s.sort()
            for t in T_s:
                reach_T.append(t)
    reach_T=list(np.round(np.array(reach_T).ravel(),2))
    #将首个服务对象到达的时刻设为0
    reach_T[0]=0
    return reach_T

In [10]:
reach_T=gen_reach_T(2.1,1.0,1000,0)
reach_T[:10]

[0, 0.47, 0.51, 0.71, 1.04, 1.06, 1.29, 1.83, 2.53, 2.67]

In [11]:
#生成对每个服务对象的服务时长
def gen_service_T(theta,gen_size,random_seed):
    np.random.seed(random_seed)
    generator=expon(0,theta)
    return list(np.round(generator.rvs(size=gen_size),2))

np.mean(gen_service_T(0.4,1000,0))

0.40155

In [12]:
#要使用pd.cut函数
import pandas as pd

In [13]:
#返回系统中的平均顾客数、队列中等待的平均顾客数及rou(平均达到率与平均服务率的比值)
def gen_Lq_Ls_rou(reach_t,service_t):
    start_service_t=[]
    stop_service_t=[]
    start_service_t.append(reach_t[0])
    stop_service_t.append(service_t[0])
    size=len(reach_t)
    for i in range(1,size):
        if reach_t[i]>=stop_service_t[i-1]:
            start_service_t.append(reach_t[i])
            stop_service_t.append(reach_t[i]+service_t[i])
        else:
            start_service_t.append(stop_service_t[i-1])
            stop_service_t.append(stop_service_t[i-1]+service_t[i])
    Lq,Ls,rou=[],[],[]
    observe_t=np.linspace(0,stop_service_t[-1],size)
    for i in range(size):
        #pd.cut()函数将区间分为左开右闭的区间，因为reach_t[0]=0,所以这里要加上1
        total_come_nums=pd.cut(reach_t,[0,observe_t[i]]).value_counts()[0]+1
        total_leave_nums=pd.cut(stop_service_t,[0,observe_t[i]]).value_counts()[0]
        sub=total_come_nums-total_leave_nums
        if sub==0:
            Lq.append(sub)
            Ls.append(sub)
        else:
            Lq.append(sub-1)
            Ls.append(sub)
        rou.append(sum(service_t)/stop_service_t[-1])
    return Lq,Ls,rou

In [14]:
import sys
#M_M_1函数输出随机模拟的各指标信息
def M_M_1(lamda=2.1,uniform_scale=1.0,theta=0.4,gen_size=2000,\
         random_seed=np.random.randint(20,size=(100,2)),bCompute_Ls_Lq_rou=False):
    
    #如果需要输出这几个指标，仅输出20个随机模拟的结果
    if bCompute_Ls_Lq_rou:random_seed=np.random.randint(20,size=(20,2))

    result=[]
    for [seed_1,seed_2] in random_seed:
        info=[]
        reach_T=gen_reach_T(lamda,uniform_scale,gen_size,seed_1)
        service_T=gen_service_T(theta,gen_size,seed_2)
        interval_T=[reach_T[i+1]-reach_T[i] for i in range(len(reach_T)-1)]
        wait_T=[0]
        for i in range(1,len(service_T)):
            temp=wait_T[i-1]+service_T[i-1]-interval_T[i-1]
            wait_T.append(temp if temp>=0 else 0)
        info.append(wait_T)
        stay_T=list(np.array(wait_T)+np.array(service_T))
        info.append(stay_T)
        if bCompute_Ls_Lq_rou:
            lq,ls,rou=gen_Lq_Ls_rou(reach_T,service_T)
            info.append(lq)
            info.append(ls)
            info.append(rou)
            Wq,Ws,Lq,Ls,rou=np.mean(np.array(info),axis=1)
            result.append([Wq,Ws,Lq,Ls,rou])
            print('In test {}:\tWq={:.3f}, Ws={:.3f}, Lq={:.3f}, Ls={:.3f}, rou={:.3f}'\
                .format(len(result),Wq,Ws,Lq,Ls,rou))
            sys.stdout.flush()
        else:
            Wq,Ws=np.mean(np.array(info),axis=1)
            result.append([Wq,Ws])
            print('In test {}:\tWq={:.3f}, Ws={:.3f}'.format(len(result),Wq,Ws))
            sys.stdout.flush()
    return np.mean(np.array(result),axis=0)

In [15]:
M_M_1()

In test 1:	Wq=2.332, Ws=2.739
In test 2:	Wq=1.543, Ws=1.949
In test 3:	Wq=1.640, Ws=2.043
In test 4:	Wq=1.418, Ws=1.809
In test 5:	Wq=1.542, Ws=1.946
In test 6:	Wq=1.751, Ws=2.144
In test 7:	Wq=2.366, Ws=2.777
In test 8:	Wq=1.515, Ws=1.906
In test 9:	Wq=4.984, Ws=5.387
In test 10:	Wq=2.353, Ws=2.767
In test 11:	Wq=2.433, Ws=2.822
In test 12:	Wq=1.804, Ws=2.206
In test 13:	Wq=1.800, Ws=2.205
In test 14:	Wq=2.352, Ws=2.754
In test 15:	Wq=1.694, Ws=2.086
In test 16:	Wq=2.300, Ws=2.700
In test 17:	Wq=1.381, Ws=1.784
In test 18:	Wq=1.811, Ws=2.217
In test 19:	Wq=2.001, Ws=2.403
In test 20:	Wq=3.105, Ws=3.516
In test 21:	Wq=1.741, Ws=2.145
In test 22:	Wq=2.088, Ws=2.479
In test 23:	Wq=2.513, Ws=2.919
In test 24:	Wq=1.874, Ws=2.279
In test 25:	Wq=1.612, Ws=2.005
In test 26:	Wq=3.435, Ws=3.837
In test 27:	Wq=1.389, Ws=1.780
In test 28:	Wq=1.770, Ws=2.161
In test 29:	Wq=2.371, Ws=2.763
In test 30:	Wq=1.640, Ws=2.034
In test 31:	Wq=2.626, Ws=3.029
In test 32:	Wq=2.871, Ws=3.282
In test 33:	Wq=1.

array([2.1663584, 2.5673604])

In [16]:
M_M_1(bCompute_Ls_Lq_rou=True)

In test 1:	Wq=1.604, Ws=2.011, Lq=3.307, Ls=4.153, rou=0.841
In test 2:	Wq=1.523, Ws=1.922, Lq=3.142, Ls=3.965, rou=0.823
In test 3:	Wq=2.357, Ws=2.768, Lq=4.991, Ls=5.864, rou=0.872
In test 4:	Wq=3.025, Ws=3.420, Lq=6.406, Ls=7.240, rou=0.834
In test 5:	Wq=1.854, Ws=2.259, Lq=3.838, Ls=4.686, rou=0.842
In test 6:	Wq=1.784, Ws=2.177, Lq=3.747, Ls=4.575, rou=0.823
In test 7:	Wq=1.891, Ws=2.292, Lq=3.977, Ls=4.822, rou=0.846
In test 8:	Wq=2.601, Ws=3.002, Lq=5.481, Ls=6.329, rou=0.845
In test 9:	Wq=3.002, Ws=3.414, Lq=6.181, Ls=7.027, rou=0.847
In test 10:	Wq=3.233, Ws=3.634, Lq=6.783, Ls=7.625, rou=0.841
In test 11:	Wq=1.697, Ws=2.090, Lq=4.412, Ls=5.412, rou=0.828
In test 12:	Wq=2.537, Ws=2.948, Lq=5.228, Ls=6.074, rou=0.845
In test 13:	Wq=1.760, Ws=2.154, Lq=3.648, Ls=4.470, rou=0.822
In test 14:	Wq=2.584, Ws=2.979, Lq=5.610, Ls=6.468, rou=0.857
In test 15:	Wq=3.142, Ws=3.554, Lq=6.598, Ls=7.462, rou=0.863
In test 16:	Wq=1.697, Ws=2.090, Lq=4.412, Ls=5.412, rou=0.828
In test 17:	Wq=1.

array([2.16085325, 2.5610255 , 4.65075   , 5.51585   , 0.83757077])

In [17]:
#例33.3

In [18]:
result=[]
test_days,test_times=2000,1000
for _ in range(test_times):
    reached_num=np.random.choice([0,1,2,3,4,5],size=test_days,\
        p=[0.23,0.3,0.3,0.1,0.05,0.02])
    can_unload=[2]*(test_days)
    delay_unload=[reached_num[0]-can_unload[0] \
        if reached_num[0]-can_unload[0]>0 else 0]
    for i in range(1,test_times):
        delay_=delay_unload[-1]+reached_num[i]-can_unload[i]
        delay_unload.append(delay_ if delay_>0 else 0)
    result.append(np.mean(delay_unload))
np.mean(result)

0.9216759999999999